# Pyvista demo in ThDSubduction

In this notebook, I showcase the usage of the Pyvista package in processing vtu dataset and generate easy-to-plot subset.

In [ ]:
import os, sys

import numpy as np
from pathlib import Path

root_path = os.path.join(Path().resolve().parent.parent.parent)

if os.path.abspath(root_path) not in sys.path:
    sys.path.append(os.path.abspath(root_path))

test_dir = os.path.join(root_path, '.test')
if not os.path.isdir(test_dir):
    os.mkdir(test_dir)

# Settings

- case directory
- pvtu step number to plot, the file name is: 
    "solution-%05d.pvtu" % pvtu_step

In [ ]:
# case name and directories
local_ThDSubduction_dir = "/mnt/lochy/ASPECT_DATA/ThDSubduction"
case_name = "chunk_geometry1/eba3d_width80_bw8000_sw2000_yd500.0_AR4"
pvtu_step = 105 # 2 or 15

if case_name is not None:
    local_dir = os.path.join(local_ThDSubduction_dir, case_name)
    assert(os.path.isdir(local_dir))

# Processing

In [ ]:
from hamageolib.research.haoyuan_3d_subduction.post_process import PYVISTA_PROCESS_THD

## First option: process the whole dataset

The simple thing to do is to load the pvtu file directly (This contains outputs at a timestep).

The downside is when processing large dataset, this might overload PC memory (The other option is the solution to this).

### Initialize the object

In [ ]:
# initiate the object
process_output = False

if process_output:
    config = {"geometry": "chunk", "Max0": 6371e3, "Min0": 3.4810e+06,\
            "Max1": 71.94572847349845*np.pi/180.0, "Max2": 80.00365006253027*np.pi/180.0, "time":0.0}
    kwargs = {"pyvista_outdir": os.path.join(test_dir, "pyvista_demo")}
    PprocessThD = PYVISTA_PROCESS_THD(os.path.join(local_dir, "output", "solution"), config, **kwargs)

### Process

In this example:
- Read the vtu file and process a list of outputs, include domain surface and domain center slices, iso-volume object of diffferent compositions as well as the points on the slab surface and plate edge surface.
- Generate a connected shape of the subductiing slab.

In [ ]:
if process_output:
    # read vtu file
    PprocessThD.read(pvtu_step)
    # slice at center
    PprocessThD.slice_center()
    # slice at surface
    PprocessThD.slice_surface()
    # slice at depth
    PprocessThD.slice_at_depth(depth=200e3)
    # extract sp_upper composition beyond a threshold
    PprocessThD.extract_iso_volume_upper(threshold=0.8)
    # extract sp_lower composition beyond a threshold
    PprocessThD.extract_iso_volume_lower(threshold=0.8)
    slab_depth = PprocessThD.get_slab_depth()
    # extract plate_edge composition beyond a threshold
    PprocessThD.extract_plate_edge(threshold=0.8)
    # extract slab surface
    PprocessThD.extract_slab_surface("sp_upper", extract_trench=True, extract_dip=True, file_type="txt")
    # extract slab edge
    PprocessThD.extract_plate_edge_surface()
    # filter the slab lower points
    PprocessThD.filter_slab_lower_points()

    print("Trench center: ", PprocessThD.trench_center)
    print("Slab depth: ", PprocessThD.slab_depth)
    print("Dip 100: ", PprocessThD.dip_100_center)

## Another option: process piece-wise

Another options is to avoid loading the pvtu file for a time step, instead, load the vtu file (For ASPECT, there are 16 each step) and process them one by one before combining the results. This will take slighter longer time, but has the virtue of less memory cost at each call of the pieces.

### Initiate the object

The only difference is we add a "n_pieces" option. This is set to 16 to match the 16 vtu outputs from ASPECT at each timestep

In [ ]:
# initiate the object
process_piecewise = True
if process_piecewise:
    n_pieces = 16
    config = {"geometry": "chunk", "Max0": 6371e3, "Min0": 3.4810e+06,\
            "Max1": 71.94572847349845*np.pi/180.0, "Max2": 80.00365006253027*np.pi/180.0, "time":0.0}
    kwargs = {"pyvista_outdir": os.path.join(test_dir, "pyvista_demo"), "n_pieces": n_pieces}
    PprocessThD = PYVISTA_PROCESS_THD(os.path.join(local_dir, "output", "solution"), config, **kwargs)

### Process piecewise

Here loop for all the pieces. For each of them, we apply a couple of operation.

One example is

    PprocessThD.process_piecewise("slice_center", ["sliced", "sliced_u"])

This means we call a member function "slice_center" of the PYVISTA_PROCESS_THD class and save the outputs to the class attribute of "sliced" and "sliced_u" (These are pyvista.DataSet, and there will be 16 instance of each).

After all pieces are processed, the function "combine_pieces" is called, where we use the "merge" function to combine the class attributes we derive from the pieces (e.g. the 16 "sliced" attributes will be merged together to a single "sliced" attribute).

Then these attributes are outputed to vtk format. Each call of the "write_key_to_file" functio will export one of the combined attributes (e.g. "sliced"), with and assigned file name (e.g. "slice_center_unbounded") and a file format ("vtp" or "vtu").

In [ ]:
# Apply the piecewise options
if process_piecewise:
    slice_depth = 200e3; r_diff=20e3
    iso_volume_threshold = 0.8
    for piece in range(n_pieces):
        PprocessThD.read(pvtu_step, piece=piece)
        PprocessThD.process_piecewise("slice_center", ["sliced", "sliced_u"])
        PprocessThD.process_piecewise("slice_surface", ["sliced_shell"])
        PprocessThD.process_piecewise("slice_at_depth", ["sliced_depth"], depth=slice_depth, r_diff=r_diff)
        PprocessThD.process_piecewise("extract_iso_volume_upper", ["iso_volume_upper"], threshold=iso_volume_threshold)
        PprocessThD.process_piecewise("extract_iso_volume_lower", ["iso_volume_lower"], threshold=iso_volume_threshold)
        PprocessThD.process_piecewise("extract_plate_edge", ["iso_plate_edge"], threshold=iso_volume_threshold)

    PprocessThD.combine_pieces()
    PprocessThD.write_key_to_file("sliced", "slice_center_unbounded", "vtp")
    PprocessThD.write_key_to_file("sliced_u", "slice_center", "vtu")
    PprocessThD.write_key_to_file("sliced_shell", "slice_outer", "vtu")
    PprocessThD.write_key_to_file("sliced_depth", "slice_depth_%.1fkm" % (slice_depth/1e3), "vtu")
    PprocessThD.write_key_to_file("iso_volume_upper", "sp_upper_above_%.2f" % (iso_volume_threshold), "vtu")
    PprocessThD.write_key_to_file("iso_volume_lower", "sp_lower_above_%.2f" % (iso_volume_threshold), "vtu")
    PprocessThD.write_key_to_file("iso_plate_edge", "plate_edge_above_%.2f" % (iso_volume_threshold), "vtu")

### Further analysis

Using the combined class attributes (Note: they are pyvista.DataSet), we can derive new variables (e.g. slab depth, trench position, e.g.).
Check that we can arrive at the same results with processing the whole dataset.

In [ ]:
if process_piecewise:
    # extract slab surface
    PprocessThD.extract_slab_surface("sp_upper", extract_trench=True, extract_dip=True, file_type="txt")
    # extract slab edge
    PprocessThD.extract_plate_edge_surface()
    # filter the slab lower points
    PprocessThD.filter_slab_lower_points()
    # get slab depth
    PprocessThD.get_slab_depth()
    
    print("Trench center: ", PprocessThD.trench_center)
    print("Slab depth: ", PprocessThD.slab_depth)
    print("Dip 100: ", PprocessThD.dip_100_center)

## Make a boundary in vtk

As part of the workflow, making a boundary as well as marker points helps to finalize plots.

One could take the boundary file and plot in Paraview with the rest of the outputs.

In [ ]:
# make domain boundary
if process_output:
    p_marker_coordinates = {"r": 6371e3 - np.arange(0, 6000e3, 1000e3), "lon": np.arange(0, 90, 10)*np.pi/180.0, "lat": np.arange(0, 50.0, 10.0)*np.pi/180.0}
    PprocessThD.make_boundary_spherical(marker_coordinates=p_marker_coordinates)

# Finalize plot in Adobe Illustrator

In this plot:

- "slice_center" outputs is used for plotting the velocity field at the center of the slab (purple vectors)
- "slice_at_depth" outputs is used for plotting the velocity field at 200 km depth (blue vectors)
- "iso_volume_sp_lower" outputs is used for plotting the slab (green)
- boundary outputs are used for plotting the grid and tick the coordinates

![](./chunk_example.png)